In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/om/user/ysa/fmri_env/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [88]:
ls atlas/

1069_fam_eur.csv                     GENUS_FS_ATLAS_DK_NANCOUNT.csv
D_atlas_covs                         GENUS_FS_ATLAS_DKT.csv
D_atlas_covs.csv                     GENUS_FS_ATLAS_DKT_NANCOUNT.csv
D_atlas_percent_missing.csv          GENUS_FS_ATLAS_D_NANCOUNT.csv
encode_cat.py                        group_makeup.py
GENUS_all_FreeSurfer_phenotypes.csv  per_site_classify/
GENUS_FS_ATLAS_D.csv*                site_makeup.csv
GENUS_FS_ATLAS_DK.csv


In [3]:
ls text_files_for_indexing/

170_columns.txt  2k_columns.txt  covariate_variables.txt  id_variables.txt


In [27]:
data = pd.read_csv('atlas/GENUS_FS_ATLAS_D.csv', low_memory=False)

columns_thickness_170 = np.genfromtxt(
    'text_files_for_indexing/170_columns.txt', 
    dtype=str, 
    delimiter='\n')

columns_volume_170 = ' '.join(
    list(columns_thickness_170)
    ).replace("_thickness_D", "_volume_D").split()

columns_2k = np.genfromtxt(
    'text_files_for_indexing/2k_columns.txt',
    dtype=str,
    delimiter='\n')

covariates = np.genfromtxt("text_files_for_indexing/covariate_variables.txt", dtype=str)

id_variables = np.genfromtxt("text_files_for_indexing/id_variables.txt", dtype=str)

def name_idx(data, l):
    return [(data.columns.get_loc(i), i) for i in l]

get = lambda cols, idx: [item[idx] for item in cols]

In [64]:
covar_idx = name_idx(data, covariates)
id_idx = name_idx(data, id_variables)
cols_thick_170_idx = name_idx(data, columns_thickness_170)
cols_vol_170_idx = name_idx(data, columns_volume_170)
cols_2k_idx = name_idx(data, columns_2k)

In [28]:
groups = data.groupby('GROUP')
control = groups.get_group('Control')
schizo = groups.get_group('Schizophrenia')
for name, group in groups:
    print name, group.shape

Bipolar Disorder (43, 2266)
Clinical High Risk (9, 2266)
Control (1286, 2266)
Familial High Risk (399, 2266)
Major Depressive Disorder (8, 2266)
Schizophrenia (912, 2266)


In [97]:
N_matrix = np.zeros((18,5), dtype=object)
groups = data.groupby('STUDY')
for idx, (name, group) in enumerate(groups):
    for i, x in enumerate((cols_thick_170_idx, cols_vol_170_idx, cols_2k_idx)):
        N_matrix[idx][0] = name
        N_matrix[idx][1] = group.dropna().shape[0]
        N_matrix[idx][i+2] = group[get(x, 0)].dropna().shape[0]
N_matrix[-1, 1:] = N_matrix[:, 1:].sum(0)
N_matrix[-1, 0] = 'sum_total'

In [98]:
pd.DataFrame(
    N_matrix,
    columns = [
        'site', 'N_all_NADroped', 
        'N_170_NADroped_t', 'N_170_NADroped_v',
        'N_2k_NADroped'
    ]
).set_index('site')

,N_all_NADroped,N_170_NADroped_t,N_170_NADroped_v,N_2k_NADroped
site,,,,
CAMH,145,194,194,194
CIDAR_P,0,46,46,46
CIDAR_VA,105,240,240,240
IMH_SIGNRP,0,302,302,302
LandR,43,74,74,74
MGH_FB12_MTH,0,88,88,88
MGH_SSS_SZB,37,101,101,101
NEFS_A,55,116,116,116
NEFS_G,4,4,4,4
